In [1]:
%%bash
dx download "UKB Metabolomics:/agarham/icd_med_dataset.tsv"

In [2]:
# Hide warnings
import warnings
warnings.filterwarnings('ignore')

# Import libraries
import pandas as pd
import numpy as np
import re

# Import the datasets
df = pd.read_csv('icd_med_dataset.tsv', sep='\t')

In [3]:
# Create list of date columns
date_cols = [column for column in df.columns if column.startswith('41280-0.')]

# Split the string by "|" and expand into separate columns
split_cols = df['41270-0.0'].str.split('|', expand=True)

# Rename columns to '41270-0.0', '41270-0.1', etc.
split_cols.columns = [f'41270-0.{i}' for i in range(split_cols.shape[1])]

# Return the splitted data to the original dataset
df[split_cols.columns] = split_cols

In [4]:
# Define the PsA codes
codes = [
    'L405',
    'M070', 'M0700', 'M0701', 'M0702', 'M0703', 'M0704', 'M0705', 'M0706', 'M0707', 'M0708', 'M0709',
    'M071', 'M0710', 'M0711', 'M0712', 'M0713', 'M0714', 'M0715', 'M0716', 'M0717', 'M0718', 'M0719',
    'M072', 'M0720', 'M0721', 'M0722', 'M0723', 'M0724', 'M0725', 'M0726', 'M0727', 'M0728', 'M0729',
    'M073', 'M0730', 'M0731', 'M0732', 'M0733', 'M0734', 'M0735', 'M0736', 'M0737', 'M0738', 'M0739'
]


# Get all 41270 and 41280 columns
cols_41270 = [col for col in df.columns if col.startswith('41270-0.')]
cols_41280 = [col for col in df.columns if col.startswith('41280-0.')]

# Initialize new columns
for code in codes:
    df[code] = 0
    df[f'{code}_date'] = np.nan

# Iterate over rows
for idx, row in df.iterrows():
    for col_41270 in cols_41270:
        value = row[col_41270]
        if value in codes:
            # Mark code presence
            df.at[idx, value] = 1

            # Get corresponding date column
            suffix = col_41270.split('41270-0.')[-1]
            corresponding_col_41280 = f'41280-0.{suffix}'
            
            if corresponding_col_41280 in df.columns:
                df.at[idx, f'{value}_date'] = row[corresponding_col_41280]

In [6]:
# Convert date columns to datetime
date_cols = [f'{code}_date' for code in codes]

for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Create a boolean df where code columns == 1
code_bool = df[codes] == 1

# psa = 1 if any code column is 1, else 0
df['psa'] = code_bool.any(axis=1).astype(int)

# Create a mask to keep dates only where corresponding code == 1, else NaT
masked_dates = df[date_cols].where(code_bool.values)

# psa_date = earliest date (min) among the masked dates per row
df['psa_date'] = masked_dates.min(axis=1)

In [7]:
# Check the number of PsA diagnoses
print(df['psa'].value_counts())
print(df['psa_date'].count())

0    500351
1      1615
Name: psa, dtype: int64
1615


In [8]:
# Check how many rows have value 0 in L405 column, while any of the other columns contains 1 in that row.
target_col = 'L405'

# Condition 1: target_col == 0
cond_target_0 = df[target_col] == 0

# Condition 2: all other codes == 1
other_cols = [col for col in codes if col != target_col]
cond_others_1 = (df[other_cols] == 1).any(axis=1)

# Rows where target_col is 0 and any others are 1
result_rows = df[cond_target_0 & cond_others_1]

# Count them
count = result_rows.shape[0]

print(f"Number of rows where '{target_col}' is 0 but others have at least one 1: {count}")

Number of rows where 'L405' is 0 but others have at least one 1: 6


In [9]:
# Select columns starting with 'M07'
m07_cols = [code for code in codes if code.startswith('M07')]

# Count how many 'M07' columns are 1 per row
count_ones = (df[m07_cols] == 1).sum(axis=1)

# Filter rows with two or more 1s in 'M07' columns
rows_multiple_ones = df[count_ones >= 2]

# Number of such rows
count = rows_multiple_ones.shape[0]

print(f"Number of rows with two or more 1s in columns starting with 'M07': {count}")

Number of rows with two or more 1s in columns starting with 'M07': 492


In [10]:
# Check the number of PsA diagnoses
print(df['L405'].value_counts())
print(df['L405_date'].count())
print(df['M073'].value_counts())
print(df['M073_date'].count())
print(df['M0739'].value_counts())
print(df['M0739_date'].count())

0    500357
1      1609
Name: L405, dtype: int64
1609
0    500719
1      1247
Name: M073, dtype: int64
1247
0    501517
1       449
Name: M0739, dtype: int64
449


In [11]:
# Slice and save the output
psa = df[['eid', 'psa', 'psa_date']]
psa.to_csv('psa_diagnoses_v2.tsv', sep='\t', index=False)

In [12]:
%%bash
dx upload "psa_diagnoses_v2.tsv"

ID                                file-J1VxxX0JZ8jYFq1Y2Xb0861q

Class                             file

Project                           project-J0yb91QJZ8jZ9ppFk6zGQgGp

Folder                            /

Name                              psa_diagnoses.tsv

State                             closing

Visibility                        visible

Types                             -

Properties                        -

Tags                              -

Outgoing links                    -

Created                           Tue Jul  1 13:12:40 2025

Created by                        agarham

 via the job                      job-J1Vvz08JZ8jkFzKbfV8fJVQV

Last modified                     Tue Jul  1 13:12:41 2025

Media type                        

archivalState                     "live"

cloudAccount                      "cloudaccount-dnanexus"
